In [2]:
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes
from datetime import datetime, timedelta
import base64

# Generate RSA keys
private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
public_key = private_key.public_key()

# To track processed timestamps
processed_timestamps = set()

# Alice's actions
def sign_message(message):
    timestamp = datetime.utcnow().isoformat()
    message_with_timestamp = f"{message}|{timestamp}"
    signature = private_key.sign(
        message_with_timestamp.encode(),
        padding.PKCS1v15(),
        hashes.SHA256()
    )
    return message_with_timestamp, base64.b64encode(signature).decode()

# Bob's actions
def verify_message(message_with_timestamp, signature_b64):
    try:
        # Extract message and timestamp
        message, timestamp = message_with_timestamp.rsplit('|', 1)
        timestamp = datetime.fromisoformat(timestamp)

        # Check if the timestamp is within the acceptable window
        current_time = datetime.utcnow()
        if current_time - timestamp > timedelta(minutes=5):
            return False, "Message timestamp is outdated"

        # Check if the timestamp was already processed
        if timestamp in processed_timestamps:
            return False, "Replay detected: Timestamp already used"

        # Verify the signature
        signature = base64.b64decode(signature_b64)
        public_key.verify(
            signature,
            message_with_timestamp.encode(),
            padding.PKCS1v15(),
            hashes.SHA256()
        )

        # Mark the timestamp as processed
        processed_timestamps.add(timestamp)
        return True, "Message is valid"
    except Exception as e:
        return False, str(e)

# Demonstration
message = "Transfer $1000 to Oscar"

# Alice signs the message
signed_message, signature = sign_message(message)
print("Signed Message:", signed_message)
print("Signature:", signature)

# Bob verifies the first message
is_valid, result = verify_message(signed_message, signature)
print("Verification Result (First Attempt):", result)

# Bob tries to verify the same message again (replay attempt)
is_valid, result = verify_message(signed_message, signature)
print("Verification Result (Replay Attempt):", result)

Signed Message: Transfer $1000 to Oscar|2024-11-20T19:03:32.029989
Signature: M7ARvpCjPFTjTIyywV10B+WRAcqPw8gKQPjeAtKduHN5EFPpdReoOVdL8P86uOgnXrzoJ7g2iOHicb3YPOQsKyTYU1FxeLZONZimU/KIx1BrE4J0U+LHiLanXn8eGhCx3j9LVXF41zUhlIxBS07m/ZObDwsKUceJlBxZj1+cfQ8TkgttwEWdsM2Tc3Jb0EprIyJ4wlrZOav59pzg/w0+IgRAhaQokk+oUESLdlcPrQDzRzxmzGJglSq4cSpU06CEuEaa2Lt11yAGq7XAQUtQHvvbSVqA0GNYHz7lKmKXTzgrchgCK6GbxaqKKdbKhUL0fUsPdaMpGSmpjdRKAw0G2Q==
Verification Result (First Attempt): Message is valid
Verification Result (Replay Attempt): Replay detected: Timestamp already used


In [3]:
import hmac
import hashlib
import os

# Shared secret key
shared_key = b'supersecretkey'

# To track used nonces
used_nonces = set()

# Alice's actions
def generate_nonce():
    return os.urandom(16).hex()

def create_mac(message, nonce):
    message_with_nonce = f"{message}|{nonce}".encode()
    mac = hmac.new(shared_key, message_with_nonce, hashlib.sha256).hexdigest()
    return message_with_nonce.decode(), mac

# Bob's actions
def verify_mac(message_with_nonce, mac):
    try:
        # Extract message and nonce
        message, nonce = message_with_nonce.rsplit('|', 1)

        # Check if the nonce has already been used
        if nonce in used_nonces:
            return False, "Replay detected: Nonce already used"

        # Verify MAC
        computed_mac = hmac.new(shared_key, message_with_nonce.encode(), hashlib.sha256).hexdigest()
        if computed_mac != mac:
            return False, "Invalid MAC"

        # Mark the nonce as used
        used_nonces.add(nonce)
        return True, "Message is valid"
    except Exception as e:
        return False, str(e)

# Demonstration
nonce = generate_nonce()
message = "Transfer $1000 to Oscar"

# Alice creates the MAC
message_with_nonce, mac = create_mac(message, nonce)
print("Message with Nonce:", message_with_nonce)
print("MAC:", mac)

# Bob verifies the first message
is_valid, result = verify_mac(message_with_nonce, mac)
print("Verification Result (First Attempt):", result)

# Bob tries to verify the same message again (replay attempt)
is_valid, result = verify_mac(message_with_nonce, mac)
print("Verification Result (Replay Attempt):", result)

Message with Nonce: Transfer $1000 to Oscar|287914b1c532323f256144f377860e08
MAC: a2057195aa8c42b31e938acfac021f0e72b93ea5969e491637b3691012eac8a4
Verification Result (First Attempt): Message is valid
Verification Result (Replay Attempt): Replay detected: Nonce already used
